In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd
import pyreadr
import torch
from IPython.core.display import display
from torch import sigmoid

from astral import LocationInfo
from astral.sun import sun
from pytorch_lightning import LightningDataModule
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from warML.sklearn.utils import RandomizedCV, get_best_model

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%load_ext autoreload
%autoreload 2
output_cols = ['date_cible','perimetre_prod','type_prod','prev_FC']
USELESS_COLS_BY_TYPE = dict(pv=['cubic_norm','angle'],
                            eolien=['t2m','ssrd','neb'])



In [ ]:
meteo_ce_allvars_ecmwf = pyreadr.read_r('meteo_ce_allvar_grille_ecmwf.RDS')[None]
prod = pyreadr.read_r('prod.RDS')[None]

station_df = pd.read_csv('liste_stations.csv', sep=';')
substation_df = pd.read_csv('coord_substations.csv')

meteo_locale_allvars_ecmwf = pyreadr.read_r('meteo_locale_allvar_grille_ecmwf.RDS')[None]
meteo_locale_vent_mf = pyreadr.read_r('meteo_locale_vent_grille_meteofrance.RDS')[None]

meteo_ce_nebtemp_mf = pyreadr.read_r('meteo_ce_nebtemp_station_meteofrance.RDS')[None]
meteo_ce_vent_mf = pyreadr.read_r('meteo_ce_vent_grille_meteofrance.RDS')[None]

In [ ]:
substation_df.rename(columns=dict(SUBSTATION='perimetre_prod'),inplace=True)
substation_df

In [ ]:
station_df

In [ ]:
def date_to_time_in_year(date:datetime):
    return (date - datetime(date.year, month=1,day=1)) / (datetime(date.year+1, month=1,day=1) - datetime(date.year, month=1,day=1) )

prod['time_in_day'] = prod.date_cible.apply(lambda dt: (3600*dt.hour+60*dt.minute + dt.second)/(24*3600))
prod['time_in_year'] = prod.date_cible.apply(date_to_time_in_year)

# Météo

In [ ]:
def compute_norm_and_angle(df):
    df['cubic_norm'] = np.sqrt(df.u100 ** 2 + df.v100 **2)
    df['angle'] = np.arctan2(df.v100, df.u100)
    if 'neb' in df.columns:
        df['neb'] = df['neb'] / 8
    return df.drop(columns=['u100','v100','ff100'])

## Météo ECMWF

In [ ]:
meteo_ce_allvars_ecmwf.rename(columns={'zone':'perimetre_prod'},inplace=True)
meteo_ce_allvars_ecmwf = meteo_ce_allvars_ecmwf[meteo_ce_allvars_ecmwf.perimetre_prod!='France']
meteo_ce_allvars_ecmwf = compute_norm_and_angle(meteo_ce_allvars_ecmwf)

meteo_ce_allvars_ecmwf
# no missing value

In [ ]:
meteo_locale_allvars_ecmwf = compute_norm_and_angle(meteo_locale_allvars_ecmwf)

meteo_locale_allvars_ecmwf.info()


## Météo France

In [ ]:
#

meteo_ce_allvars_mf = meteo_ce_nebtemp_mf.merge(meteo_ce_vent_mf, how='left', on=['date_lancement','date_cible','zone','echeance'])
meteo_ce_allvars_mf.rename(columns={'zone':'perimetre_prod'},inplace=True)
meteo_ce_allvars_mf = meteo_ce_allvars_mf[meteo_ce_allvars_mf.perimetre_prod!='France']

meteo_ce_allvars_mf= compute_norm_and_angle(meteo_ce_allvars_mf)
meteo_ce_allvars_mf # missing u100, v100, f100

In [ ]:
meteo_locale_vent_mf= compute_norm_and_angle(meteo_locale_vent_mf)
meteo_locale_vent_mf.info()

## Merge meteos

In [ ]:
meteo_ce_merged = meteo_ce_allvars_mf.merge(meteo_ce_allvars_ecmwf, how='inner', on=['date_cible','date_lancement','perimetre_prod','echeance'])
meteo_ce_merged.info()

In [ ]:
meteo_local_merged = meteo_locale_allvars_ecmwf.merge(meteo_locale_vent_mf, how='inner', on=['date_cible','date_lancement','perimetre_prod','echeance'])
meteo_local_merged.info()

# Modèles nationaux


## National Preprocessing

In [ ]:
def keep_latest_date_lancement(df):
    # Keep valid previsions (date_lancement < veille, 16h)
    # Keep latest date_lancement for each date_cible

    out = df.groupby(['perimetre_prod', 'date_cible']).last().reset_index()
    return out

In [ ]:
meteo_ce_merged.info()

In [ ]:
valid_meteo_df = keep_latest_date_lancement(meteo_ce_merged)
valid_meteo_df = valid_meteo_df.drop(columns=['date_lancement']).pivot_table(index=['date_cible','echeance'],columns='perimetre_prod')

valid_meteo_df = valid_meteo_df.reset_index()
valid_meteo_df.info()

In [ ]:
national_df = prod[prod.perimetre_prod=='national'].merge(valid_meteo_df, on='date_cible', how='inner')
national_df.info()


In [ ]:
class DataModule(LightningDataModule):

    def __init__(self,df:pd.DataFrame, type_prod:str,batch_size:int):

        super().__init__()
        self.df = df
        self.type_prod=type_prod
        self.batch_size=batch_size

    def prepare_data(self):
        df = self.df[self.df.type_prod==self.type_prod].drop(columns=['perimetre_prod','type_prod','date_lancement','puissance_installee','comptage'],errors='ignore')
        # DROP USELESS COLS
        for useless_c in USELESS_COLS_BY_TYPE[self.type_prod]:
            df = df[df.columns.drop(list(df.filter(regex=useless_c)))]

        df_train = df[df.date_cible.dt.year < 2019].dropna()
        df_test = df[df.date_cible.dt.year == 2019]

        std_scaler = StandardScaler()
        self.df_test = df_test
        self.df_train = df_train
        self.features = list(df_train.drop(columns=['FC','date_cible']).columns)
        self.x_train = torch.from_numpy(std_scaler.fit_transform(df_train.drop(columns=['FC','date_cible']))).float()
        self.x_test = torch.from_numpy(std_scaler.transform(df_test.drop(columns=['FC','date_cible']))).float()
        self.y_train = torch.from_numpy(df_train['FC'].values).float()
        self.y_test = torch.from_numpy(df_test['FC'].values).float()




def predict(model, x):
    model.eval()
    with torch.no_grad():
        out = model(x)
        return sigmoid(out).numpy()



## National PV +wind

In [ ]:
models =     [
        ('XGB',
                  GradientBoostingRegressor(),
                  dict(n_estimators=[100],
                       learning_rate=[1e-1],
                       max_depth=[3]))]
                  #('PyGAM',
                   #LinearGAM(),
                  # dict(terms=term_list)
                   #)]
outs = []

In [ ]:


national_preds = pd.DataFrame(columns=output_cols)

for type_prod in tqdm(['pv','eolien'], desc="Type of prod"):
    dm = DataModule(national_df, type_prod=type_prod, batch_size=32)
    dm.prepare_data()

    out = RandomizedCV(dm.x_train, dm.y_train,
                                            scoring='neg_mean_squared_error',
                                            models=models,
                                            n_iter=1,
                       return_preds=True)
    outs.append(out)
    best_model = get_best_model(out, metric='score')
    test_df = dm.df_test
    train_df = dm.df_train

    test_df['prev_FC'] = best_model.predict(dm.x_test)
    test_df['type_prod'] =type_prod
    test_df['perimetre_prod'] ='national'
    national_preds = pd.concat([national_preds,test_df[output_cols]])



In [ ]:
%%time

meteo_local_merged = keep_latest_date_lancement(meteo_local_merged)
local_df = prod[prod.perimetre_prod!='national'].merge(meteo_local_merged, on=['date_cible','perimetre_prod'], how='inner')
local_df.info()

In [ ]:
local_preds = pd.DataFrame(columns=output_cols)

for perimetre, type_prod in tqdm([('A.COM','eolien'), ('CXSSE','eolien'), ('HOSTE','pv'),('ONDES','pv')],desc="Prod type"):
    
    dm = DataModule(local_df[local_df.perimetre_prod==perimetre], type_prod=type_prod, batch_size=32)
    dm.prepare_data()

    out = RandomizedCV(dm.x_train, dm.y_train, scoring='neg_mean_squared_error',
                                            models=models,
                                           n_iter=1,
                       return_preds=True)

    outs.append(out)
    best_model = get_best_model(out,metric="score")
    test_df = dm.df_test
    test_df['prev_FC'] = best_model.predict(dm.x_test)
    test_df['type_prod'] =type_prod
    test_df['perimetre_prod'] =perimetre
    local_preds = pd.concat([local_preds,test_df[output_cols]])


# Postprocessing

In [ ]:
# Clip FC
overall_preds = pd.concat([national_preds, local_preds])
overall_preds.prev_FC = overall_preds.prev_FC.clip(lower=0,upper=1)
overall_preds = overall_preds.merge(substation_df, on='perimetre_prod', how='left')
overall_preds.date_cible = pd.to_datetime(overall_preds.date_cible, utc=True)

overall_preds.groupby(['perimetre_prod','type_prod']).count()

In [ ]:
def zero_fc_if_dark(row):
    if row.type_prod !='pv':
        return row
    s = add_sunset_sunrise(row)
    if row.date_cible < s['sunrise']  or row.date_cible  > s['sunset'] :
        row.prev_FC =0
    return row

def add_sunset_sunrise(row):
    # Don't use lat, lon to zero
    if row.perimetre_prod == "national":

        sunrise = row.date_cible.replace(hour=5)
        sunset = row.date_cible.replace(hour=20)
    # Use sunrise, sunset hours to zero PV
    else:
        loc = LocationInfo(latitude=row.LAT, longitude=row.LON)
        s = sun(loc.observer, date=row.date_cible, tzinfo=row.date_cible.tzinfo)
        sunrise, sunset = s['sunrise'], s['sunset']

    return dict(sunset=sunset, sunrise=sunrise)

# Zero FC at night for pv
overall_preds = overall_preds.apply(zero_fc_if_dark, axis=1)
overall_preds




In [ ]:

name = 'preds/das_gute_model_alexandre_rozier.csv'
overall_preds.drop(columns=['LAT','LON']).to_csv(name,sep=';', index=False, date_format='%Y-%m-%dT%H:%M:%SZ',decimal='.')


# TODO
- ~~Remove PV prod at night~~
- ~~Center + scale variables~~
- ~~time in the day~~
- ~~clip pv according to sunrise/sunset information~~
- ~~add time in the year~~
- hourly models ? -> use more linear models for that
- weather correction model
- closest mf station
- smooth temp
- ~~neural net for wind power~~
- add RMSE / MAPE / error to nns metrics
- add prod of previous week ? allowed ?
- ~~**Save output, check it & plot -> add to warML ?**~~

### Notes:
- echeance seems to help the model take into account uncertainty in forecasts


# Explo predictions
Importance de variables

In [ ]:
pv_hoste = overall_preds[(overall_preds.type_prod=='pv') & (overall_preds.perimetre_prod=='HOSTE')]
pv_hoste[['sunset','sunrise']] = pv_hoste.apply(add_sunset_sunrise,axis=1, result_type='expand')
pv_hoste

In [ ]:
import plotly.express as px
dm = DataModule(national_df, type_prod='pv', batch_size=32)
dm.prepare_data()
pv_bm = outs[0].best_models['XGB']

px.bar(y=pv_bm.feature_importances_, x=dm.features)

In [ ]:
dm = DataModule(national_df, type_prod='eolien', batch_size=32)
dm.prepare_data()
pv_bm = outs[1].best_models['XGB']

px.bar(y=pv_bm.feature_importances_, x=dm.features)

In [ ]:
for i, (perimetre, type_prod) in enumerate(tqdm([('A.COM','eolien'), ('CXSSE','eolien'), ('HOSTE','pv'),('ONDES','pv')],desc="Prod type")):

    dm = DataModule(local_df[local_df.perimetre_prod==perimetre], type_prod=type_prod, batch_size=32)
    dm.prepare_data()
    bm = outs[i+2].best_models['XGB']

    display(px.bar(y=bm.feature_importances_, x=dm.features,title=f'{perimetre} - {type_prod}'))